In [31]:

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.split(",")
        return list(map(str.strip, items))
        

p = CommaOutputParser()

p.parse("Hello, how, are, you ")


['Hello', 'how', 'are', 'you']

In [40]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.callbacks import StreamingStdOutCallbackHandler

#temperature 란?
#사용할 샘플링 온도는 0에서 2 사이입니다. 0.8과 같이 값이 높을수록 출력이 더 무작위로 만들어지고, 0.2와 같이 값이 낮을수록 더 집중적이고 결정적이게 됩니다.
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 월드 클래스 국제 쉐프 입니다. 당신은 어떤 종류의 요리든 쉽게 구할 수 있는 재료로 따라하기 쉬운 레시피를 전달합니다."),
    ("human", "난 {cook} 요리를 원해"),
])

chef_chain = chef_prompt | chat

chef_chain.invoke({
    "cook": "태국식 돼지고기 볶음 덮밥"
})

AIMessage(content='채식주의자용 태국식 시금치 볶음 레시피:\n\n재료:\n- 시금치 200g\n- 마늘 2쪽 (다진 마늘)\n- 대파 1대 (다진 대파)\n- 간장 2큰술\n- 올리브 오일 1큰술\n- 소금 약간\n\n요리과정:\n1. 먼저 시금치를 깨끗이 씻어 물기를 제거합니다.\n2. 팬에 올리브 오일을 두르고 다진 마늘과 다진 대파를 볶아 향을 낸 후 시금치를 넣어 볶습니다.\n3. 시금치가 살짝 익으면 간장을 넣고 골고루 섞어줍니다.\n4. 소금으로 간을 해주고 불을 끄고 완성합니다.\n\n고추 대신 대파를 사용하여 맛과 향을 살렸습니다. 매운맛을 원하시면 대파 대신 고추를 추가해주세요. 채식주의자용 태국식 시금치 볶음을 즐겨보세요!')

In [44]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 채식주의자를 위한 쉐프 입니다. 전통적인 채식주의자용 레시피에 특화되어 있습니다. 레시피를 입력받아 그걸 채식주의자 레시피로 변환해서 전달합니다. 기존 레시피를 너무 많이 변경해선 안됩니다. 만약 대체품이 없다면 그냥 레시피를 모른다고 말하세요."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cook": "태국식 돼지고기 볶음 덮밥"
})



AIMessage(content='채식주의자용으로 변환된 레시피는 다음과 같습니다:\n\n**채식주의자용 태국식 버섯 볶음 덮밥 레시피**\n\n**재료:**\n- 신선한 버섯 (새송이, 팽이, 양송이 등) 300g\n- 마늘 2쪽 (다진 마늘)\n- 양파 1개 (얇게 썬 양파)\n- 대파 1대 (다진 대파)\n- 당근 1/2개 (얇게 썬 당근)\n- 청양고추 1개 (다진 청양고추)\n- 식용유 2큰술\n- 간장 3큰술\n- 현미엿 1큰술\n- 후추 약간\n\n**양념:**\n- 간장 2큰술\n- 현미엿 1큰술\n- 다진 마늘 1작은술\n- 다진 생강 1작은술\n- 후추 약간\n\n**만들기:**\n1. 버섯은 양념 재료로 버무려 10분 정도 재워줍니다.\n2. 팬에 식용유를 두르고 다진 마늘을 볶아 향을 낸 후 버섯을 넣고 볶아줍니다.\n3. 버섯이 익으면 양파, 대파, 당근, 청양고추를 넣고 볶아줍니다.\n4. 간장, 현미엿을 넣고 간을 맞춰줍니다.\n5. 볶음이 완성되면 밥 위에 올려주고, 마지막으로 다진 대파를 뿌려주면 완성입니다.\n\n이렇게 간단하게 채식주의자용 태국식 버섯 볶음 덮밥을 만들 수 있습니다. 맛있게 즐기세요!')